# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Student 1 - __[student1@ulb.ac.be](mailto:student1@ulb.ac.be) - Student ID 123456__
### Student 2 - __[student2@vub.ac.be](mailto:student2@ulb.ac.be) - Student ID 789012__
### Student 3 - __[student3@ulb.ac.be](mailto:student3@ulb.ac.be) - Student ID 345678__

### Video presentation: www.youtube.com/abcd1234

## Project Title


# Introduction


In [1]:
#install.packages("e1071")
#install.packages("tidyverse")
#install.packages("pillar")
#install.packages("caret")

#install.packages("drat", repos="https://cran.rstudio.com")
#drat:::addRepo("dmlc")
#install.packages("xgboost", repos="http://dmlc.ml/drat/", type = "source")

In [34]:
library(rpart)
library(e1071)
library(rpart.plot)
library(randomForest)
library(nnet)
library(xgboost)

In [3]:
sub_format <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Submission_format.csv", header=TRUE)
set_val_test <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv", header=TRUE)
set_labels <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", header=TRUE)
dataset <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv", header=TRUE)

In [4]:
choices <- length(unique(set_labels$status_group))
choices

[1] 3

In [5]:
dim(set_labels)

#apply(is.na(set_val),2,any)

[1] 59400     2

In [6]:
dataset$funder[0:40]#Problem mac, don't have na values

[1] "Roman"                    "Grumeti"                 
 [3] "Lottery Club"             "Unicef"                  
 [5] "Action In A"              "Mkinga Distric Coun"     
 [7] "Dwsp"                     "Rwssp"                   
 [9] "Wateraid"                 "Isingiro Ho"             
[11] "Private"                  "Danida"                  
[13] "World Vision"             "Lawatefuka Water Supply" 
[15] "Biore"                    "Rudep"                   
[17] "Unicef"                   "Unicef"                  
[19] "Hesawa"                   "Danida"                  
[21] "Twe"                      "Dwsp"                    
[23] "Unicef"                   "Isf"                     
[25] "African Development Bank" "Government Of Tanzania"  
[27] "Sobodo"                   "Hesawa"                  
[29] "Government Of Tanzania"   "Lawatefuka Water Supply" 
[31] "Water"                    "Private Individual"      
[33] "Government Of Tanzania"   "Undp"                    
[35] ""                         "Not Known"               
[37] "Danida"                   "Kirde"                   
[39] "Cefa"                     "Government Of Tanzania"

In [12]:
data1<-replace(dataset,dataset=='',NA)#Replaced to NA
#data1[20:40]
dim(na.omit(data1))

[1] 27813    40

'''date_recorded
num_private
funder
installer
wpt_name
subvillage
region? (code 27, region 21, why?)
lga
ward
recorded_by
scheme_name
payment
quantity
waterpoint_type_group'''

In [13]:
cleaning <- c("date_recorded", "num_private", "funder", "installer", "wpt_name", "subvillage", "lga", "ward", 
              "region_code", "scheme_name", "payment", "quantity", "waterpoint_type_group", "recorded_by")
X<-dataset[,setdiff(colnames(dataset),cleaning)]
X<-replace(X,X=='',NA)#Replaced to NA
#X[20:26]

In [14]:
choices <- length(unique(X$construction_year))
choices
choices2 <- unique(X$construction_year)
choices2

[1] 55

[1] 1999 2010 2009 1986    0 2011 1987 1991 1978 1992 2008 1974 2000 2002 2004
[16] 1972 2003 1980 2007 1973 1985 1970 1995 2006 1962 2005 1997 2012 1996 1977
[31] 1983 1984 1990 1982 1976 1988 1989 1975 1960 1961 1998 1963 1971 1994 1968
[46] 1993 2001 1979 1967 2013 1969 1981 1964 1966 1965

In [15]:
#distric code 0 ?? a demander
#population 0 -> mean ???
#public_meeting -> plus frequent
#scheme_management -> frequent
#permit -> plus frequent
#construction_year -> int(mean)

#dim(data.frame(X$permit))

replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

table(X$permit)
X$permit[is.na(X$permit)] <- tail(names(sort(table(X$permit))), 1)
table(X$permit)

table(X$scheme_management)
X$scheme_management[is.na(X$scheme_management)] <- tail(names(sort(table(X$scheme_management))), 1)
table(X$scheme_management)

table(X$public_meeting)
X$public_meeting[is.na(X$public_meeting)] <- tail(names(sort(table(X$public_meeting))), 1)
table(X$public_meeting)


False  True 
17492 38852 


False  True 
17492 41908 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            36793 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


         Company             None            Other       Parastatal 
            1061                1              766             1680 
Private operator              SWC            Trust              VWC 
            1063               97               72            40670 
 Water authority      Water Board              WUA              WUG 
            3153             2748             2883             5206 


False  True 
 5055 51011 


False  True 
 5055 54345 

In [16]:
X

id,amount_tsh,gps_height,longitude,latitude,basin,region,district_code,population,public_meeting,⋯,management,management_group,payment_type,water_quality,quality_group,quantity_group,source,source_type,source_class,waterpoint_type
<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
69572,6000,1390,34.93809,-9.85632177,Lake Nyasa,Iringa,5,109,True,⋯,vwc,user-group,annually,soft,good,enough,spring,spring,groundwater,communal standpipe
8776,0,1399,34.69877,-2.14746569,Lake Victoria,Mara,2,280,True,⋯,wug,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe
34310,25,686,37.46066,-3.82132853,Pangani,Manyara,4,250,True,⋯,vwc,user-group,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple
67743,0,263,38.48616,-11.15529772,Ruvuma / Southern Coast,Mtwara,63,58,True,⋯,vwc,user-group,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple
19728,0,0,31.13085,-1.82535885,Lake Victoria,Kagera,1,0,True,⋯,other,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe
9944,20,0,39.17280,-4.76558728,Pangani,Tanga,8,1,True,⋯,vwc,user-group,per bucket,salty,salty,enough,other,other,unknown,communal standpipe multiple
19816,0,0,33.36241,-3.76636472,Internal,Shinyanga,3,0,True,⋯,vwc,user-group,never pay,soft,good,enough,machine dbh,borehole,groundwater,hand pump
54551,0,0,32.62062,-4.22619802,Lake Tanganyika,Shinyanga,3,0,True,⋯,wug,user-group,unknown,milky,milky,enough,shallow well,shallow well,groundwater,hand pump
53934,0,0,32.71110,-5.14671181,Lake Tanganyika,Tabora,6,0,True,⋯,vwc,user-group,never pay,salty,salty,seasonal,machine dbh,borehole,groundwater,hand pump


In [17]:
X$construction_year[X$construction_year==0] <- NA
X$construction_year[is.na(X$construction_year)] <- mean(X$construction_year,na.rm=T)
X$construction_year <- as.integer(X$construction_year)

In [18]:
X$longitude[X$longitude==0] <- NA
X$longitude[is.na(X$longitude)] <- mean(X$longitude,na.rm=T)

#X$latitude[X$latitude==0] <- NA
#X$latitude[is.na(X$latitude)] <- mean(X$latitude,na.rm=T)
sum(X$latitude==-0.00000002)

[1] 1812

In [19]:
X$permit[X$permit=='True'] <- 1
X$permit[X$permit=='False'] <- 0

X$permit <- as.integer(X$permit)

X$public_meeting[X$public_meeting=='True'] <- 1
X$public_meeting[X$public_meeting=='False'] <- 0

X$public_meeting <- as.integer(X$public_meeting)

In [21]:
summary(X)

       id          amount_tsh         gps_height       longitude    
 Min.   :    0   Min.   :     0.0   Min.   : -90.0   Min.   :29.61  
 1st Qu.:18520   1st Qu.:     0.0   1st Qu.:   0.0   1st Qu.:33.35  
 Median :37062   Median :     0.0   Median : 369.0   Median :35.15  
 Mean   :37115   Mean   :   317.7   Mean   : 668.3   Mean   :35.15  
 3rd Qu.:55656   3rd Qu.:    20.0   3rd Qu.:1319.2   3rd Qu.:37.18  
 Max.   :74247   Max.   :350000.0   Max.   :2770.0   Max.   :40.35  
    latitude          basin              region          district_code  
 Min.   :-11.649   Length:59400       Length:59400       Min.   : 0.00  
 1st Qu.: -8.541   Class :character   Class :character   1st Qu.: 2.00  
 Median : -5.022   Mode  :character   Mode  :character   Median : 3.00  
 Mean   : -5.706                                         Mean   : 5.63  
 3rd Qu.: -3.326                                         3rd Qu.: 5.00  
 Max.   :  0.000                                         Max.   :80.00  
   pop

In [22]:
names(X)

[1] "id"                    "amount_tsh"            "gps_height"           
 [4] "longitude"             "latitude"              "basin"                
 [7] "region"                "district_code"         "population"           
[10] "public_meeting"        "scheme_management"     "permit"               
[13] "construction_year"     "extraction_type"       "extraction_type_group"
[16] "extraction_type_class" "management"            "management_group"     
[19] "payment_type"          "water_quality"         "quality_group"        
[22] "quantity_group"        "source"                "source_type"          
[25] "source_class"          "waterpoint_type"

In [23]:
#one hot converstion
#install.packages('dummies')
library(dummies)


variables_to_keep<-names(X)
X_onehot <- dummy.data.frame(X[,variables_to_keep], sep="_")

dummies-1.5.6 provided by Decision Patterns


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 

In [24]:
#names(data_factor_onehot)
X_onehot

,id,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,⋯,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,69572,6000,1390,34.93809,-9.85632177,0,1,0,0,0,⋯,1,0,0,0,1,0,0,0,0,0
2,8776,0,1399,34.69877,-2.14746569,0,0,0,0,1,⋯,0,1,0,0,1,0,0,0,0,0
3,34310,25,686,37.46066,-3.82132853,0,0,0,0,0,⋯,0,1,0,0,0,1,0,0,0,0
4,67743,0,263,38.48616,-11.15529772,0,0,0,0,0,⋯,1,0,0,0,0,1,0,0,0,0
5,19728,0,0,31.13085,-1.82535885,0,0,0,0,1,⋯,0,1,0,0,1,0,0,0,0,0
6,9944,20,0,39.17280,-4.76558728,0,0,0,0,0,⋯,0,0,1,0,0,1,0,0,0,0
7,19816,0,0,33.36241,-3.76636472,1,0,0,0,0,⋯,1,0,0,0,0,0,0,1,0,0
8,54551,0,0,32.62062,-4.22619802,0,0,0,1,0,⋯,1,0,0,0,0,0,0,1,0,0
9,53934,0,0,32.71110,-5.14671181,0,0,0,1,0,⋯,1,0,0,0,0,0,0,1,0,0


In [25]:
vars <- c("id", "status_group")
set_labels <- read.csv("database/Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv", header=TRUE)

labels_onehot <- dummy.data.frame(set_labels[,vars], sep="_")

names(labels_onehot)[names(labels_onehot) == "status_group_functional"] <- "functional"
names(labels_onehot)[names(labels_onehot) == "status_group_functional needs repair"] <- "functional_needs_repair"
names(labels_onehot)[names(labels_onehot) == "status_group_non functional"] <- "non_functional"
labels_onehot

#set_labels$status_group[set_labels$status_group=='functional']<-0
#set_labels$status_group[set_labels$status_group=='functional needs repair']<-0.5
#set_labels$status_group[set_labels$status_group=='non functional']<-1

#set_labels$status_group <- as.double(set_labels$status_group)
#set_labels


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”


,id,functional,functional_needs_repair,non_functional
,<int>,<int>,<int>,<int>
1,69572,1,0,0
2,8776,1,0,0
3,34310,1,0,0
4,67743,0,0,1
5,19728,1,0,0
6,9944,1,0,0
7,19816,0,0,1
8,54551,0,0,1
9,53934,0,0,1


In [26]:
Y<-labels_onehot[,setdiff(colnames(labels_onehot),"id")]
Y<-set_labels[2]
Y[Y == "functional needs repair"] <- "functional_needs_repair"
Y[Y == "non functional"] <- "non_functional"
Y

status_group
<chr>
functional
functional
functional
non_functional
functional
functional
non_functional
non_functional
non_functional


In [27]:
X_final<-X_onehot[,setdiff(colnames(X_onehot),"id")]
X_final

,amount_tsh,gps_height,longitude,latitude,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,basin_Lake Victoria,basin_Pangani,⋯,source_class_groundwater,source_class_surface,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other
,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,6000,1390,34.93809,-9.85632177,0,1,0,0,0,0,⋯,1,0,0,0,1,0,0,0,0,0
2,0,1399,34.69877,-2.14746569,0,0,0,0,1,0,⋯,0,1,0,0,1,0,0,0,0,0
3,25,686,37.46066,-3.82132853,0,0,0,0,0,1,⋯,0,1,0,0,0,1,0,0,0,0
4,0,263,38.48616,-11.15529772,0,0,0,0,0,0,⋯,1,0,0,0,0,1,0,0,0,0
5,0,0,31.13085,-1.82535885,0,0,0,0,1,0,⋯,0,1,0,0,1,0,0,0,0,0
6,20,0,39.17280,-4.76558728,0,0,0,0,0,1,⋯,0,0,1,0,0,1,0,0,0,0
7,0,0,33.36241,-3.76636472,1,0,0,0,0,0,⋯,1,0,0,0,0,0,0,1,0,0
8,0,0,32.62062,-4.22619802,0,0,0,1,0,0,⋯,1,0,0,0,0,0,0,1,0,0
9,0,0,32.71110,-5.14671181,0,0,0,1,0,0,⋯,1,0,0,0,0,0,0,1,0,0


In [28]:
N<-nrow(X_final)    #Number of examples
n<-ncol(X_final)    #Number of input variables

names(X_final)<-make.names(names(X_final))

In [29]:
X_pca<-data.frame(prcomp(X_final,retx=T)$x)

In [30]:
X_pca

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,⋯,PC150,PC151,PC152,PC153,PC154,PC155,PC156,PC157,PC158,PC159
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-5694.65598,592.626326,-186.203084,2.3411072,-1.6078650,3.1386680,-2.14335613,0.496976205,1.098876923,-0.3718124,⋯,-1.616545e-16,8.399314e-16,1.154675e-16,-1.094079e-16,2.982721e-18,-1.266122e-16,-8.708282e-17,-8.376868e-17,-3.070447e-17,1.978124e-17
2,303.63567,743.109148,-21.352283,13.0292677,3.3197694,-3.2369737,1.31350858,1.012054200,-0.650420512,0.8549088,⋯,2.143468e-16,-2.835189e-16,3.485710e-16,1.652578e-16,1.478584e-16,-2.378166e-16,-4.537661e-17,-2.264111e-16,8.053468e-17,-1.187746e-17
3,292.07535,34.510625,66.124405,12.2689017,2.6605354,-0.2701010,2.79118688,1.182500162,-0.321827684,0.2028049,⋯,-5.682304e-16,3.894454e-16,-3.745919e-16,-1.344292e-16,2.071066e-16,-2.207006e-16,5.136620e-18,6.695956e-17,-1.478895e-17,3.631927e-17
4,325.52024,-413.856576,-53.723258,-5.8008588,-57.5539431,0.8034112,-1.13208940,-0.476679276,-0.847686803,-1.1755996,⋯,-1.809533e-16,-5.965280e-16,-1.441989e-17,-6.388492e-17,-4.721245e-17,5.701019e-17,2.254550e-16,1.916314e-16,-1.778527e-18,-1.442139e-17
5,330.59903,-682.621009,-67.889513,-0.7626552,6.5200030,-4.5858135,-1.54205853,2.067721343,-0.564478420,0.2155048,⋯,1.310800e-16,-2.835189e-16,7.101524e-17,-1.018896e-16,2.369798e-16,5.405065e-17,1.346823e-16,7.066025e-17,-9.684080e-17,2.411805e-17
6,310.59983,-682.846965,-66.865744,12.8741376,-0.2025347,1.2369008,3.91634214,0.002843854,-0.819028696,-0.9050292,⋯,7.180671e-16,9.194034e-16,-1.704474e-15,-1.560997e-16,-3.313731e-17,1.667111e-16,-1.985888e-16,4.638252e-16,-1.446369e-17,-1.141680e-16
7,330.59909,-682.626383,-67.883428,-0.5602538,4.2262372,-2.0422393,-0.57065274,-0.738957002,-0.088323096,-0.8394809,⋯,-4.304283e-17,4.266336e-16,-1.861575e-16,-5.765076e-17,-1.125463e-17,-5.507472e-17,-5.215161e-17,1.270205e-16,1.997836e-17,1.938762e-17
8,330.59909,-682.626880,-67.883551,-0.5672506,4.2629810,-2.1878232,-1.50653340,-1.437341891,0.205704523,0.7416535,⋯,5.529431e-18,7.943949e-16,-4.845300e-16,8.692760e-17,1.180229e-16,2.282521e-17,6.575538e-17,3.959317e-17,-1.094754e-16,-7.934275e-18
9,330.59927,-682.633044,-67.876729,-0.3394667,1.1867450,-1.4623933,-1.89990028,-0.988586346,-0.286550443,-1.2895820,⋯,-1.000610e-15,-8.154285e-16,3.238512e-16,-1.428267e-17,-3.648130e-16,1.294565e-16,-6.049996e-17,-3.342533e-16,3.721717e-17,-1.183740e-17


In [31]:
#DS<-cbind(X_final,label=Y)
DS<-cbind(X_pca,label=Y)
names(DS)<-make.names(names(DS))
DS$status_group<-factor(DS$status_group)

In [32]:
DS

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,⋯,PC151,PC152,PC153,PC154,PC155,PC156,PC157,PC158,PC159,status_group
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,-5694.65598,592.626326,-186.203084,2.3411072,-1.6078650,3.1386680,-2.14335613,0.496976205,1.098876923,-0.3718124,⋯,8.399314e-16,1.154675e-16,-1.094079e-16,2.982721e-18,-1.266122e-16,-8.708282e-17,-8.376868e-17,-3.070447e-17,1.978124e-17,functional
2,303.63567,743.109148,-21.352283,13.0292677,3.3197694,-3.2369737,1.31350858,1.012054200,-0.650420512,0.8549088,⋯,-2.835189e-16,3.485710e-16,1.652578e-16,1.478584e-16,-2.378166e-16,-4.537661e-17,-2.264111e-16,8.053468e-17,-1.187746e-17,functional
3,292.07535,34.510625,66.124405,12.2689017,2.6605354,-0.2701010,2.79118688,1.182500162,-0.321827684,0.2028049,⋯,3.894454e-16,-3.745919e-16,-1.344292e-16,2.071066e-16,-2.207006e-16,5.136620e-18,6.695956e-17,-1.478895e-17,3.631927e-17,functional
4,325.52024,-413.856576,-53.723258,-5.8008588,-57.5539431,0.8034112,-1.13208940,-0.476679276,-0.847686803,-1.1755996,⋯,-5.965280e-16,-1.441989e-17,-6.388492e-17,-4.721245e-17,5.701019e-17,2.254550e-16,1.916314e-16,-1.778527e-18,-1.442139e-17,non_functional
5,330.59903,-682.621009,-67.889513,-0.7626552,6.5200030,-4.5858135,-1.54205853,2.067721343,-0.564478420,0.2155048,⋯,-2.835189e-16,7.101524e-17,-1.018896e-16,2.369798e-16,5.405065e-17,1.346823e-16,7.066025e-17,-9.684080e-17,2.411805e-17,functional
6,310.59983,-682.846965,-66.865744,12.8741376,-0.2025347,1.2369008,3.91634214,0.002843854,-0.819028696,-0.9050292,⋯,9.194034e-16,-1.704474e-15,-1.560997e-16,-3.313731e-17,1.667111e-16,-1.985888e-16,4.638252e-16,-1.446369e-17,-1.141680e-16,functional
7,330.59909,-682.626383,-67.883428,-0.5602538,4.2262372,-2.0422393,-0.57065274,-0.738957002,-0.088323096,-0.8394809,⋯,4.266336e-16,-1.861575e-16,-5.765076e-17,-1.125463e-17,-5.507472e-17,-5.215161e-17,1.270205e-16,1.997836e-17,1.938762e-17,non_functional
8,330.59909,-682.626880,-67.883551,-0.5672506,4.2629810,-2.1878232,-1.50653340,-1.437341891,0.205704523,0.7416535,⋯,7.943949e-16,-4.845300e-16,8.692760e-17,1.180229e-16,2.282521e-17,6.575538e-17,3.959317e-17,-1.094754e-16,-7.934275e-18,non_functional
9,330.59927,-682.633044,-67.876729,-0.3394667,1.1867450,-1.4623933,-1.89990028,-0.988586346,-0.286550443,-1.2895820,⋯,-8.154285e-16,3.238512e-16,-1.428267e-17,-3.648130e-16,1.294565e-16,-6.049996e-17,-3.342533e-16,3.721717e-17,-1.183740e-17,non_functional


In [33]:
#model<- rpart(status_group~.,data=DS, method = "class") ### Fill with your code here
#model<- svm(formula=status_group~.,data=DS, type="C-classification") ### Fill with your code here
#model1<- lm(label.functional~.,data=as.data.frame(DS))

# nnet

In [107]:
accuracyFromConfusionMatrix <- function(confusion_matrix){
    if(dim(confusion_matrix)[1] < 2){ 
       if(rownames(confusion_matrix) == "nonspam"){
           confusion_matrix <- rbind(confusion_matrix,c(0,0))
           rownames(confusion_matrix)[2] <- "spam"
       }
       if(rownames(confusion_matrix) == "spam"){
           confusion_matrix <- rbind(c(0,0),confusion_matrix)
           rownames(confusion_matrix)[1] <- "nonspam"
       }
    }
    return((confusion_matrix[1,1]+confusion_matrix[2,2])/sum(confusion_matrix))
}


computeResults <- function(Y,Y_hat,explanatory_string="",verbose=TRUE){
    
    confusion_matrix <- table(Y_hat,Y)
    accuracy <- accuracyFromConfusionMatrix(confusion_matrix)
    misclassification_rate <- 1 - accuracy
    
    if(verbose){
        print(paste("[INFO] - Confusion matrix",explanatory_string,":"))
        print(confusion_matrix)
        print(paste("[INFO] - Accuracy",explanatory_string,":",accuracy))
        print(paste("[INFO] - Misclassification rate",explanatory_string,":",misclassification_rate))
    }
    
    return(accuracy)
}

In [168]:
#test without cross validation

CV_folds <- 10

size_CV <-floor(N/CV_folds)
    
idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
X_ts<-X_final[idx_ts,]  
Y_ts<-Y[idx_ts,]
Y_onehot<-labels_onehot[idx_ts,]

idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
X_tr<-X_final[idx_tr,]
Y_tr<-Y[idx_tr,]         

DS<-cbind(X_tr,status_group=Y_tr)
DS$status_group<-factor(DS$status_group)
    
    
model<- nnet(status_group~.,data=DS, size=10, maxit=1000, trace=T, MaxNWts=2000)#1000 max iterations ?
Y_hat_ts<- predict(model,X_ts)

# weights:  1633
initial  value 51603.875911 
iter  10 value 46749.159022
iter  20 value 46684.393602
iter  30 value 46667.040480
iter  40 value 46644.659238
iter  50 value 46589.027797
iter  60 value 46526.439676
iter  70 value 46514.053222
iter  80 value 46424.464947
iter  90 value 46119.632171
iter 100 value 45970.373087
iter 110 value 45820.716698
iter 120 value 45750.142986
iter 130 value 45535.600324
iter 140 value 45487.429544
iter 150 value 44970.785245
iter 160 value 44670.284079
iter 170 value 43814.326071
iter 180 value 42156.941755
iter 190 value 40690.805229
iter 200 value 40095.327701
iter 210 value 38867.155012
iter 220 value 38229.265915
iter 230 value 37803.389587
iter 240 value 37572.673472
iter 250 value 37401.674303
iter 260 value 36970.092242
iter 270 value 36827.589552
iter 280 value 36740.168205
iter 290 value 36668.381425
iter 300 value 36608.918153
iter 310 value 36559.135308
iter 320 value 36519.774670
iter 330 value 36453.690206
iter 340 value 36388.240245
it

In [169]:
function(x){
   (x-min(x))/(max(x)-min(x))
}

Y_hat <- t(apply(Y_hat_ts, 1, function(x)(x-min(x))/(max(x)-min(x))))
                 
Y_hat[,1] <- ifelse(Y_hat[,1] < 0.99,0,1)
Y_hat[,2] <- ifelse(Y_hat[,2] < 0.99,0,1)
Y_hat[,3] <- ifelse(Y_hat[,3] < 0.99,0,1)
Y_hat
Y_onehot

function(x){
   (x-min(x))/(max(x)-min(x))
}

,functional,functional_needs_repair,non_functional
53461,1,0,0
53462,0,0,1
53463,1,0,0
53464,0,0,1
53465,1,0,0
53466,1,0,0
53467,1,0,0
53468,1,0,0
53469,1,0,0
53470,1,0,0


,id,functional,functional_needs_repair,non_functional
,<int>,<int>,<int>,<int>
53461,59971,1,0,0
53462,31784,0,0,1
53463,43834,0,0,1
53464,294,0,0,1
53465,67986,1,0,0
53466,62645,1,0,0
53467,6296,1,0,0
53468,55249,0,0,1
53469,26264,1,0,0


In [170]:
Y_onehot<-Y_onehot[,setdiff(colnames(Y_onehot),"id")]
threshold <- 0.5
Y_hat <- ifelse(Y_hat_ts[,1] > threshold,"functional",ifelse(Y_hat_ts[,2] > threshold,"functional_needs_repair","non_functional"))
data.matrix(Y_hat)

53461,functional
53462,non_functional
53463,functional
53464,non_functional
53465,functional
53466,functional
53467,functional
53468,non_functional
53469,functional
53470,non_functional
53471,non_functional


In [183]:
#table(data.frame(Y_hat),data.frame(Y))

require(caret)
#acc <- confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))$overall['Accuracy']
confusionMatrix(as.factor(Y_hat),as.factor(Y[idx_ts,]))

Warning message in levels(reference) != levels(data):
“la taille d'un objet plus long n'est pas multiple de la taille d'un objet plus court”
Warning message in confusionMatrix.default(as.factor(Y_hat), as.factor(Y[idx_ts, :
“Levels are not in the same order for reference and data. Refactoring data to match.”


Confusion Matrix and Statistics

                         Reference
Prediction                functional functional_needs_repair non_functional
  functional                    2830                     336            731
  functional_needs_repair          0                       0              0
  non_functional                 380                      84           1579

Overall Statistics
                                          
               Accuracy : 0.7423          
                 95% CI : (0.7309, 0.7533)
    No Information Rate : 0.5404          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.4963          
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: functional Class: functional_needs_repair
Sensitivity                     0.8816                        0.00000
Specificity                     0.6092                    

In [174]:
table(Y)

Y
             functional functional_needs_repair          non_functional 
                  32259                    4317                   22824 

In [43]:
#change std and error, maybe cross validating and taking the best performing one ?

#nnet model

CV_folds <- 10#10

size_CV <-floor(N/CV_folds)

CV_err<-matrix(0,nrow=1,ncol=CV_folds)


for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X_pca[idx_ts,]  
    Y_ts<-Y[idx_ts,]
    Y_onehot<-labels_onehot[idx_ts,]
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X_pca[idx_tr,]
    Y_tr<-Y[idx_tr,]         
        # Create a dataset including only the first nb_components principal components
    DS<-cbind(X_tr,status_group=Y_tr)
    DS$status_group<-factor(DS$status_group)

    # Model fit (using lm function)
    model<- nnet(status_group~.,data=DS, size=5, decay=1.0e-5, maxit=50, trace=FALSE)#1000 max iterations ?
    #model<- rpart(status_group~.,data=DS)

    # Model predict
    Y_hat_ts<- predict(model,X_ts)
    #Y_hat_ts <- dummy.data.frame(data.frame(Y_hat_ts), sep="_") #for random forest
} 
#print(paste("CV error=",round(apply(CV_err,1,mean),digits=4), " ; std dev=",round(apply(CV_err,1,sd),digits=4)))


#model<- nnet(status_group~.,data=DS, size=3, decay=1.0e-5, maxit=50)
#Y_hat_ts<- predict(model,X_pca)

[1] "CV error= 0.502  ; std dev= 0.0398"


In [39]:
Y_hat_ts

,functional,functional_needs_repair,non_functional
1,0.7669169,0.07903475,0.1540484
2,0.6553190,0.06727934,0.2774016
3,0.6553190,0.06727934,0.2774016
4,0.4527964,0.07713807,0.4700655
5,0.4527964,0.07713807,0.4700655
6,0.4527964,0.07713807,0.4700655
7,0.4527964,0.07713807,0.4700655
8,0.4527964,0.07713807,0.4700655
9,0.4527964,0.07713807,0.4700655
10,0.4527964,0.07713807,0.4700655


# XGBoost


The downloaded binary packages are in
	/var/folders/__/7lz1mlfd6dn6kkhv3mc87zv40000gn/T//RtmpClFxWf/downloaded_packages


Warning message:
“unable to access index for repository http://dmlc.ml/drat/src/contrib:
  La ligne commençant par '<!DOCTYPE html>...' est incorrecte !”
Warning message:
“package ‘xgboost’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


# Test feature seletion (inconclusive)

In [101]:
labels_onehot[,4]

[1] 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
   [37] 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 1 1 1
   [73] 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 1
  [109] 1 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0
  [145] 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 1
  [181] 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 1
  [217] 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0
  [253] 1 0 0 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0
  [289] 0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1
  [325] 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0
  [361] 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0
  [397] 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 1 1 1 1 0 0 1
  [433] 0 0 0 1 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0
  [469] 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 1 1
  [505] 1 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1
  [541] 1 0 1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1
  [577] 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 1
  [613] 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 1 0 0
  [649] 1 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0
  [685] 0 1 0 0 0 1 1 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0
  [721] 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0
  [757] 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0
  [793] 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 1 0 1 1
  [829] 1 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1
  [865] 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0
  [901] 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 0 0
  [937] 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 0 0
  [973] 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0
 [1009] 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0
 [1045] 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1
 [1081] 1 1 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0
 [1117] 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1
 [1153] 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0
 [1189] 1 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 1 1 0
 [1225] 0 0 1 0 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0
 [1261] 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 1 1
 [1297] 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0
 [1333] 0 1 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0
 [1369] 1 0 0 0 1 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0
 [1405] 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 1 0 0 1 0 0 1 0
 [1441] 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 1
 [1477] 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0
 [1513] 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0
 [1549] 1 1 1 0 1 0 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 [1585] 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 1
 [1621] 0 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 0 0 1 0
 [1657] 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 1 1 0 1
 [1693] 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 0 0 1 1
 [1729] 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 0
 [1765] 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0
 [18

In [113]:
#First model

CV_folds <- 2#10

size_CV <-floor(N/CV_folds)

n<-20
CV_err<-matrix(0,nrow=n,ncol=CV_folds)

#labels_onehot for MSE


for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X_pca[idx_ts,]  
    Y_ts<-Y[idx_ts,]
    Y_onehot<-labels_onehot[idx_ts,]
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X_pca[idx_tr,]
    Y_tr<-Y[idx_tr,]         
     
    for (nb_components in 1:n) {
        # Create a dataset including only the first nb_components principal components
        DS<-cbind(X_tr[,1:nb_components,drop=F],status_group=Y_tr)
        DS$status_group<-factor(DS$status_group)
        
        # Model fit (using lm function)
        model<- randomForest(status_group~.,data=DS)
        #model<- rpart(status_group~.,data=DS)
        
        # Model predict
        Y_hat_ts<- predict(model,X_ts[,1:nb_components,drop=F])
        Y_hat_ts <- dummy.data.frame(data.frame(Y_hat_ts[]), sep="_") #for random forest
        
        #Pour commencer un MSE
        CV_err[nb_components,i]<-mean((Y_hat_ts[,1]-Y_onehot[,2])^2+(Y_hat_ts[,2]-Y_onehot[,3])^2+(Y_hat_ts[,3]-Y_onehot[,4])^2)
        
    }
} 
#Y_hat_ts #<- dummy.data.frame(data.frame(Y_hat_ts[]), sep="_")
print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV_err,1,mean),digits=4), " ; std dev=",round(apply(CV_err,1,sd),digits=4)))

Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts argument ignored”
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
“non-list contrasts ar

 [1] "#Features:  1  ; CV error= 0.9338  ; std dev= 0.0035" 
 [2] "#Features:  2  ; CV error= 0.7482  ; std dev= 0.0057" 
 [3] "#Features:  3  ; CV error= 0.7031  ; std dev= 0.0021" 
 [4] "#Features:  4  ; CV error= 0.5724  ; std dev= 0.0076" 
 [5] "#Features:  5  ; CV error= 0.5441  ; std dev= 0.0083" 
 [6] "#Features:  6  ; CV error= 0.5189  ; std dev= 0.0073" 
 [7] "#Features:  7  ; CV error= 0.511  ; std dev= 0.0117"  
 [8] "#Features:  8  ; CV error= 0.4931  ; std dev= 0.0082" 
 [9] "#Features:  9  ; CV error= 0.4881  ; std dev= 0.0073" 
[10] "#Features:  10  ; CV error= 0.4749  ; std dev= 0.0082"
[11] "#Features:  11  ; CV error= 0.4568  ; std dev= 0.008" 
[12] "#Features:  12  ; CV error= 0.4533  ; std dev= 0.002" 
[13] "#Features:  13  ; CV error= 0.4485  ; std dev= 0.0018"
[14] "#Features:  14  ; CV error= 0.4452  ; std dev= 0.002" 
[15] "#Features:  15  ; CV error= 0.4445  ; std dev= 0.0046"
[16] "#Features:  16  ; CV error= 0.4488  ; std dev= 0.0023"
[17] "#Features:  17  ; 

In [115]:
#rpart 13 features max avec pca 10-fold, mieux 0.47
#random forest 12 features pas mal, min error en 15 features mais pas en standard deviation

In [114]:
print(model)


Call:
 randomForest(formula = status_group ~ ., data = DS) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 4

        OOB estimate of  error rate: 22.19%
Confusion matrix:
                        functional functional_needs_repair non_functional
functional                   13802                     487           1858
functional_needs_repair       1113                     666            374
non_functional                2560                     197           8643
                        class.error
functional                0.1452282
functional_needs_repair   0.6906642
non_functional            0.2418421


In [ ]:
Y_hat<- predict(model,X_final)
data.frame(Y_hat)

        
#empirical_error<-mean((Y_hat-Y)^2) ### Fill with your code here

#print(paste("Empirical error=",round(empirical_error,digits=4)))

#(1-0.67656858)^2+(0-0.077980476)^2+(0-0.2454509)^2

In [ ]:
data.frame(model$variable.importance)# for rpart

In [ ]:
#install.packages("rpart.plot")
prp(model)
rpart.plot(model)

In [ ]:
# plot tree
plot(model, uniform=TRUE,
   main="Classification Tree - Rpart")
text(model, use.n=TRUE, all=TRUE, cex=.8)

In [ ]:
CV_folds <- 10

size_CV <-floor(N/CV_folds)

CV_err<-matrix(0,nrow=n,ncol=CV_folds)

for (i in 1:CV_folds) {
    
    idx_ts<-(((i-1)*size_CV+1):(i*size_CV))  ### idx_ts represents the indices of the test set the i-th fold
    X_ts<-X[idx_ts,]  
    Y_ts<-Y[idx_ts]  
     
    idx_tr<-setdiff(1:N,idx_ts) ### idx_tr represents  indices of the training sefor the i-th fold
    X_tr<-X[idx_tr,]
    Y_tr<-Y[idx_tr]                          
    
    # Compute correlation across all the input variables and the target variable
    correlation_vector <-abs(cor(X_tr,Y_tr))
    # Rank variables according to correlation with the output
    correlation_ranking_idx <-sort(correlation_vector,dec=T,index.return=T)$ix
     
    for (nb_features in 1:n) {
        # Create a dataset including only the nb_features most correlated variables with the output
        DS<-cbind(X_tr[,correlation_ranking_idx[1:nb_features],drop=F],imdb_score=Y_tr)
        
        # Model fit (using lm function)
        model<- lm(imdb_score~.,DS)
        
        # Model prediction
        Y_hat_ts<- predict(model,X_ts[,correlation_ranking_idx[1:nb_features],drop=F])
        
        # Cross validation error = MSE
        CV_err[nb_features,i] <- mean((Y_hat_ts-Y_ts)^2)
    }
}  

print(paste("#Features: ",c(1:n)," ; CV error=",round(apply(CV_err,1,mean),digits=4), " ; std dev=",round(apply(CV_err,1,sd),digits=4)))

# Data preprocessing

## Missing value imputation

## Feature engineering

## Feature selection



# Model selection

## Model 1

## Model 2

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


# Alternative models





# Conclusions